# 使用jupyter notebook 实现决策树


## 调包实现决策树

In [121]:
#使用sklearn-data 
# data
from sklearn.model_selection import train_test_split
def create_data():
    iris = load_iris()
    df = pd.DataFrame(iris.data, columns=iris.feature_names)
    df['label'] = iris.target
    df.columns = [
        'sepal length', 'sepal width', 'petal length', 'petal width', 'label'
    ]
    data = np.array(df.iloc[:100, [0, 1, -1]])
    # print(data)
    return data[:, :2], data[:, -1]


X, y = create_data()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [126]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
import grahviz
clf = DecisionTreeClassifier()
clf.fit(X_train,y_train)

ModuleNotFoundError: No module named 'grahviz'

In [124]:
clf.score(X_test, y_test)

0.8

In [ ]:
#可视化
tree_pic = export_graphviz(clf, out_file="mytree.pdf")
with open('mytree.pdf') as f:
    dot_graph = f.read()

## ID3算法，利用信息增益构建决策树
一、特征选择使用信息增益
二、构建树
三、剪枝

In [ ]:
"""整理需要用到的函数
1. 数据集获取函数（）: 输出训练集和测试集合；
2. 训练集与测试集标准化
3. 信息增益计算函数（训练集）：
4. 获取最大信息增益的特征函数（训练集合）：特征维度index
5. 递归建树（训练集）：字典树
6. 计算模型性能（字典树头节点，测试集）
"""

In [3]:
# 倒入需要的库以及数据集
import pandas as pd 
import numpy as np
from sklearn.datasets import load_boston,load_iris
import os 

pwd = os.path.abspath(os.path.__file__)
iris = load_iris()
boston = load_boston()

In [4]:
# 构建规划器对训练集标准化
from sklearn.preprocessing import StandardScaler
std_scaler = StandardScaler()
def scaler(features,tool):
    tool.fit(features)
    std_features = tool.transform(features)
    return std_features

In [5]:
# 香农熵计算函数
def calent(dataframe):
    num = dataframe.shape[0]
    labels = dataframe.iloc[:,-1].value_counts()
    p = labels/num
    ent = sum(-p*np.log2(p))
    return ent

In [6]:
# 构造测试用例，测试函数
def createDataset():
    testcase = {'a':[1,1,0,0,1],
                'b':[0,0,0,1,1],
                'labels':['yes','no','yes','yes','no']}
    testdf = pd.DataFrame(testcase)
    return testdf

testcase = createDataset()
ent = calent(testcase)
ent

0.9709505944546686

In [7]:
# 选择最大信息增益
def infoGain(dataframe):
    dflen = dataframe.shape[0]
    baseEnt = calent(dataframe)
    maxent = 0
    axis = -1
    for i in range(dataframe.shape[1]-1):
        curent = 0
        cats = dataframe.iloc[:,i].value_counts().index
        for j in cats:
            subdf = dataframe[dataframe.iloc[:,i] == j]
            tempent = calent(subdf)
            curent += (subdf.shape[0]/dflen)*tempent
        finalEnt = baseEnt - curent
#         print('第',i,'列的信息增益为',finalEnt)
        if (finalEnt>maxent):
            maxent = finalEnt
            axis = i
    return i             

In [127]:
a = pd.DataFrame({'a':[1,1,1,0],'b':[1,0,0,1],'c':[0,1,1,0],'lab':['yes','no','yes','no']})
a['b']

0    1
1    0
2    0
3    1
Name: b, dtype: int64

In [9]:
axis = infoGain(a)

In [111]:
#按照给定列进行数据帧切分
"""
函数功能：按照给定列划分数据集
输入：
    df:数据帧
    axis:要切割的列
    value: 
输出：子数据帧
"""
def splitdf(dataframe,axis,value):
    col = dataframe.columns[axis]
    # print(dataframe[col]==0)

    """选择与value匹配的行,在子数据帧中把列切掉
    0    False
    1     True
    2     True
    3    False
    """
    subdf = dataframe.loc[dataframe[col] == value,:].drop(col,axis=1)
    return subdf

In [112]:
splitdf(a,2,0)

,a,b,lab
0,1,1,yes
3,0,1,no


In [12]:

"""
函数说明： 使用ID3算法递归构建决策树，递归先判断停止条件
输入：
    训练数据集D，特征集A阈值
输出:
    字典形式的树T
1）若D中所有实例属于一类，则T为单节点树，将C作为该节点类的标记，返回T
2）若特征集为空，则T为单节点树，并将D中实例最大的类C作为该节点的类标记，返回T
3）否则，递归计算信息增益，选择信息增益最大的列将D切割为若干非空子集D，将Di中实例数最大的类作为标记，构建子节点。由节点与其子节点
   构成树T
4）对第I个子节点，以Di为训练集，A-Ai为特征集调用1）-3），得到子树
"""

'\n函数说明： 使用ID3算法递归构建决策树，递归先判断停止条件\n输入：\n    训练数据集D，特征集A阈值\n输出:\n    字典形式的树T\n1）若D中所有实例属于一类，则T为单节点树，将C作为该节点类的标记，返回T\n2）若特征集为空，则T为单节点树，并将D中实例最大的类C作为该节点的类标记，返回T\n3）否则，递归计算信息增益，选择信息增益最大的列将D切割为若干非空子集D，将Di中实例数最大的类作为标记，构建子节点。由节点与其子节点\n   构成树T\n4）对第I个子节点，以Di为训练集，A-Ai为特征集调用1）-3），得到子树\n'

In [13]:
def buildDecisionTree(dataset):
#     print(dataset)
    cols = list(dataset.columns[:-1])
    labels = dataset.iloc[:,-1].value_counts()
    
    #只有一类特征或数据集只剩下一行
    if dataset.shape[0] == 1 or len(cols)==1:
        return labels[0]
    else:
        #只有一类
        if labels.shape[0] == 1:
            return labels[0]
        axis = infoGain(dataset)
        #注意定义内部变量来保存列名
        feature = cols[axis]
        nodeDict = {feature:{}}
        del cols[axis]
        cats = set(dataset.iloc[:,axis].value_counts().index)
        print(cats)
        for i in cats:
#             print('i:',i,'axis:',axis)
            subdf = splitdf(dataset,axis,i)
#             print(subdf)
            nodeDict[feature][i] = buildDecisionTree(subdf)
#             print(nodeDict)
        return nodeDict 

In [14]:
mytree = buildDecisionTree(a)

{0, 1}
{1}
{0}


In [15]:
"""
函数说明：将决策树保存为npy文件
输入：保存路径，树
输出：无
"""
file = '/Users/laurent/Desktop/学习/Staticstics_Lihang/CH05'
def saveTree(path,treeDict):
    np.save(os.path.join(path,'ID3Tree.npy'),treeDict)
    print('文件保存到',os.path.join(path))
    
def loadTree(path):
    tree = np.load(os.path.join(path,'ID3Tree.npy')).item()
    return tree

In [16]:
#使用决策树进行分类
"""函数功能： 对一个测试实例进行分类
输入： inputree：决策树
      labels：决策树特征类别序号
      test：测试样本
输出:  testRes:list 测试样本类别列表
      
"""

'函数功能： 对一个测试实例进行分类\n输入： inputree：决策树\n      labels：决策树特征类别序号\n      test：测试样本\n输出:  testRes:list 测试样本类别列表\n      \n'

## C4.5 算法，利用信息增益比建树

In [17]:
#构建测试集
test = {'bred':[1,0,1,0,1,0],'suit':[0,0,1,0,1,0],'glasses':[1,0,1,1,0,1],'label':['no','no','yes','no','yes','no']}
test = pd.DataFrame(test)
test['label'].value_counts().values
sub = test.iloc[:,2].value_counts()
-sub*sub* np.log2(sub)

1   -32.0
0    -4.0
Name: glasses, dtype: float64

In [102]:
# 计算信息增益
"""
函数说明： 计算数据集的熵
输入： dataframe
输出： 熵
"""
def entropycal(df):
    labels = df['label'].value_counts().values
    num = df.shape[0]
    p = labels/num
    ent = -np.sum(p * np.log2(p))
    return ent
"""
函数说明：计算训练样本每一个特征的信息增益
输入： dataframe
"""
def mulinfocal(df):
    baseent = entropycal(df)
    cats_num = len(df.columns)-1
    totalnum = df.shape[0]
    info = []
    #异常判断
    for i in range(cats_num):
        ent = 0
        cats = df.iloc[:,i].value_counts().index
        for j in cats:
            subdf = df[df.iloc[:,i] == j]
            ent += (subdf.shape[0]/totalnum )*entropycal(subdf)
        info.append(baseent-ent)
    return info

mulinfocal(test)

"""
函数说明： 计算训练样本每一个特征的信息增益比
"""
def inforatecal(df):
    infolist = df
    totalnum = int(df.shape[0])
    cats_num = len(df.columns)-1
    infolist = mulinfocal(df)
    hlist = []
    for i in range(cats_num):
        diffvalues = df.iloc[:,i].value_counts()
        nums = diffvalues.shape[0]
#         print(diffvalues/totalnum)
        p = diffvalues/totalnum
#         print(p)
        finalh = np.sum(-p*p*np.log2(p), axis=0)
#         print(finalh)
        hlist.append(finalh)
#     print(hlist)
#     print(infolist)
    # 对两个列表每一个元素进行操作
    inforate = list(map((lambda x:x[0]/x[1]),zip(infolist, hlist)))
#     inforate = infolist ./ hlist     
    idx = inforate.index(max(inforate))
    return idx
# 验证函数

In [114]:
test.

0     no
1     no
2    yes
3     no
4    yes
5     no
Name: label, dtype: object

In [115]:
# 测试C4.5 的
def BuilDTree_C4(dataset):
#     print(dataset)
    cols = list(dataset.columns[:-1])
    labels = dataset.iloc[:,-1].value_counts()
    
    #只有一类特征或数据集只剩下一行
    if dataset.shape[0] == 1 or len(cols)==1:
        return labels.index[0]
    else:
        #只有一类
        if labels.shape[0] == 1:
            return labels[0]
        axis = inforatecal(dataset)
        #注意定义内部变量来保存列名
        feature = cols[axis]
        nodeDict = {feature:{}}
        del cols[axis]
        cats = set(dataset.iloc[:,axis].value_counts().index)
        print(cats)
        for i in cats:
#             print('i:',i,'axis:',axis)
            subdf = splitdf(dataset, axis, i)
#             print(subdf)
            nodeDict[feature][i] = buildDecisionTree(subdf)
#             print(nodeDict)
        return nodeDict 

In [116]:
buildDecisionTree(test)

{0, 1}
{0, 1}
{0, 1}


{'glasses': {0: {'suit': {0: 1, 1: 1}}, 1: {'suit': {0: 3, 1: 1}}}}

In [108]:
# 
# 选择信息增益比最大的列进行切分
# def splitdf(df, idx):
#     col = cols[idx]
#     subdf = test.drop(columns=col)
#     return subdf
# # splitdf(test)
# test.label.value_counts()

In [ ]:
#构建树结构搭建决策树
"""
考虑好节点结构，以及特征，特征idx的存储，切分后的子集的遍历存储
"""
class Node:
    def __init__(self, root=True, label=True, Feature_name=None. feature=None)
        self.root = root 
        self.label = label
        self,feature_name = feature_name
        self.feature = feature
        self.tree = {}
        self.result = {
            'label': self.label,
            'feature' : self.feature,
            'tree' : self.tree
        }
    def __repr__(self):
        return '{}'.format(self.result)
    
    def add_node(self, val, node):
        self.tree[val] = node
    
    def predic(self, features):
        if self.root is True:
            return self.label
        return self.tree[features[self.feature]].predict(features)

class DTree:
    def __init__(self, epsilon=0.1):
        self.epsilon = epsilon
        self.tree = {}
    

## CART 算法，利用基尼指数建树

In [ ]:
# 基尼指数算法计算最佳切分特征
"""
统计数据帧中label的种类与对应的数量，计算概率p， gini = 1-sum((pi)*2)
表示集合的不确定性，gini越大，不确定性越高
在二叉分类树中，只做二分类的gini计算，按二分类形式，对该特征该值进行树的构建；
"""
def ginical(df):
    

### 决策树剪枝算法

#### 1.计算节点的损失函数
#### 2. 树剪枝
#### 3. 遍历新树进行剪枝

In [129]:
"""
函数说明：遍历一个树的所有节点，直到叶子节点计算损失函数， 不好实现
输入：根节点

"""
def getrootloss(traindf, inputTree):
    firstStr = list(inputTree.keys())[0]
    secodDict = inputTree[firstStr]
    total_num = 0
    for key in list(secondDict.keys()):
        if isTree(secondDict[key]):
            #继续递归
            
    cols = traindf.colums()
    feature_idx = col.index(firstStr)
    cats = traindf[firstStr].value_counts().values.tolist()
    totalinfo = 0.0
    for cat in cats:
        subdf = traindf[traindf[firstStr] == cat]
        subdfnum = subdf.shape[0]
        labels = subdf.iloc[-1].value_counts()
        p = labels.iloc[0]/subdfnum
        info = -1* np.sum(p*np.log2(p))
        totalinfo += info
    return totalinfo
def info(df):
    return calent(df)

def get_gt(rootloos, leafent, nodesnum):
    gt = (ent - getrootloss)/ (nodesnum-1)

def splitDataSet(df, col_idx, value):
    subdf = df[df.iloc[:, col_idx] == value]
    return subdf

def pruneTree(inputTree, traindf, testdf):
    
    """
    输入： 决策树，训练集
    输出： 剪枝子树列表
    获取树的key，递归计算，判断字典子集是否是树，如果是的话就继续递归切分（数据集和测试集也要切分），直到子树都是叶子节点，计算分类错误率，
        如果分类错误率降低，那么替换树，变为节点（多数分类规则进行归类），返回类别就可。
    """
    
    firstStr = list(inputTree.keys())[0]
    secondDict = inputTree[firstStr]        # 获取子树
    num_cats = len(secondDict)
    totalinfo = 0.0
    labels = traindf.iloc[:, -1]
#     featKey = copy.deepcopy(firstStr)  
#     labelIndex = labels.index(featKey)  
#     subLabels = copy.deepcopy(labels)
#     del(labels[labelIndex])  
    for key in list(secondDict.keys()):  
        if isTree(secondDict[key]):
            # 深度优先搜索,递归剪枝
            subDataSet = splitDataSet(dataSet,labelIndex,key)
            subTestSet = splitDataSet(testData,labelIndex,key)
            if len(subDataSet) > 0 and len(subTestSet) > 0:
                inputTree[firstStr][key] = pruneTree(secondDict[key],subDataSet,subTestSet,copy.deepcopy(labels))
    if calcTestErr(inputTree,testData,subLabels) < testMajor(majorityCnt(classList),testData):
        # 剪枝后的误差反而变大，不作处理，直接返回
        return inputTree 
    else:
        # 剪枝，原父结点变成子结点，其类别由多数表决法决定
        return majorityCnt(classList)

IndentationError: expected an indented block (<ipython-input-129-3af7a3c4d1bd>, line 12)